<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/repair_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))
sys.path.append(os.path.join(REPO_DIR, "logic"))
from importnb import Notebook

with Notebook():
    import FireBase as fb
    import user_controller as uc

from datetime import datetime



In [4]:
def start_repair(sensor: str):
    """
    Start the repair process for a given sensor.
    Logs the start time and updates the fault status in Firebase.
    """
    code, fault = fb.get_fault(sensor)
    if code != 200 or fault is None:
        return f"Fault '{sensor}' not found."

    now_iso = datetime.now().isoformat()
    fb.update_fault(sensor, {
        "repair_start_time": now_iso,
        "status": "Repair Started"
    })
    return f"Repair for '{sensor}' started at {now_iso}."

def calculate_fault_score(fault):
    """
    Calculates a score for repairing a fault based on its severity and time taken.

    Args:
        fault (dict): Fault data including severity and repair start time.

    Returns:
        int: Score awarded for repairing the fault.
    """
    severity_multipliers = {
        "Low": 1,
        "Medium": 2,
        "High": 3
    }

    severity = fault.get("severity", "Low")
    multiplier = severity_multipliers.get(severity, 1)

    try:
        start_time = datetime.fromisoformat(fault["repair_start_time"])
        end_time = datetime.now()
        minutes_passed = max(1, (end_time - start_time).total_seconds() / 60)
    except Exception:
        minutes_passed = 1

    score = round(multiplier * (60 / minutes_passed))
    return max(1, score)


def complete_repair(sensor: str):
    """
    Completes the repair process for a fault:
    - Retrieves the currently logged-in user.
    - Calculates XP.
    - Updates fault status in Firebase.
    - Updates the user's XP.

    Args:
        sensor (str): The ID of the sensor to complete repair for.

    Returns:
        str: Summary message about the repair and score.
    """
    code, user = uc.get_current_user()
    if code != 200:
        return "❌ No user is logged in."

    username = user["username"]

    code, fault = fb.get_fault(sensor)
    if code != 200 or fault is None:
        return f"❌ Fault '{sensor}' not found."

    score = calculate_fault_score(fault)

    # Mark fault as repaired and assign it to the current user
    fb.mark_fault_as_repaired(sensor, username)

    # Update user's XP in Firebase
    code, response = fb.update_user_score(username, score)
    if code != 200:
        return f"❌ Failed to update user score for '{username}'."

    # Retrieve updated user data to show new score
    code, user_data = fb.get_user(username)
    if code != 200 or user_data is None:
        return f"✅ Fault repaired, but failed to retrieve updated user score."

    new_score = user_data.get("score", "?")
    return f"✅ Fault '{sensor}' repaired by {username}. +{score} XP (Total: {new_score})."
